In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
import langchain

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
KEY="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"

In [6]:
llm= ChatOpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=KEY,
)


C:\Users\Ahmed\AppData\Local\Temp\ipykernel_29396\1116653725.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm= ChatOpenAI(


In [7]:
# llm = ChatOpenAI(
#     api_key=KEY,
#     model="gpt-3.5-turbo",
#     temperature=0.5
# )


In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2":{"mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
        },
        "3":{"mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"

        }    
}        

In [10]:
tmplt = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
Create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""


In [11]:
 quiz_gen_prompt=PromptTemplate(input_variables=["text","number","subject","tone","response_json"],template=tmplt)

In [12]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_gen_prompt, output_key="quiz", verbose=True)

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_29396\2809350267.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_gen_prompt, output_key="quiz", verbose=True)


In [13]:
tmplt2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity\
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
 quiz_evt_prompt=PromptTemplate(input_variables=["subject","quiz",],template=tmplt2)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evt_prompt, output_key="review", verbose=True)

In [16]:
gen_evt_chain=SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"], verbose=True)

In [17]:
file_path=r"C:\Users\Ahmed\MiniRAG\data.txt"

In [18]:
with open(file_path,"r")as file:
    txt=file.read()

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=3
SUBJECT="EGYPT"
TONE="hard"

In [21]:
with get_openai_callback() as cb:
    response=gen_evt_chain(
        {
            "text":txt,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json": json.dumps(RESPONSE_JSON)

            
        }
    )

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_29396\1304170320.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=gen_evt_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Egypt, is a country spanning the northeast corner of Africa and southwest corner of Asia via the Sinai Peninsula. It is bordered by the Mediterranean Sea to the north, the Gaza Strip of Palestine and Israel to the northeast, the Red Sea to the east, Sudan to the south, and Libya to the west; the Gulf of Aqaba in the northeast separates Egypt from Jordan and Saudi Arabia. Cairo is the capital, largest city, and leading cultural center, while Alexandria is the second-largest city and an important hub of industry and tourism.[20] With over 107 million inhabitants, Egypt is the third-most populous country in Africa and 15th-most populated in the world.

Egypt has one of the longest histories of any country, tracing its heritage along the Nile Delta back to the 6thâ€“4th millennia BCE. Considered a cradle of civilisation, Ancient Egypt saw some of the earliest developments of writing

In [22]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1444
Prompt Tokens:1171
Completion Tokens:273
Total Cost:0.0023025


In [23]:
response

{'text': 'Egypt, is a country spanning the northeast corner of Africa and southwest corner of Asia via the Sinai Peninsula. It is bordered by the Mediterranean Sea to the north, the Gaza Strip of Palestine and Israel to the northeast, the Red Sea to the east, Sudan to the south, and Libya to the west; the Gulf of Aqaba in the northeast separates Egypt from Jordan and Saudi Arabia. Cairo is the capital, largest city, and leading cultural center, while Alexandria is the second-largest city and an important hub of industry and tourism.[20] With over 107 million inhabitants, Egypt is the third-most populous country in Africa and 15th-most populated in the world.\n\nEgypt has one of the longest histories of any country, tracing its heritage along the Nile Delta back to the 6thâ€“4th millennia BCE. Considered a cradle of civilisation, Ancient Egypt saw some of the earliest developments of writing, agriculture, urbanisation, organised religion and central government.[21] Egypt was an early an

In [24]:
quiz=response.get("quiz")

In [25]:
quiz=json.loads(quiz)

In [26]:
quiz_table_data = []
for key, value in quiz.items():
   mcq = value["mcq"]
   options = " | ".join(
       [
           f"{option}: {option_value}"
           for option, option_value in value["options"].items()
       ]
   )
   
   correct = value["correct"]
   quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [27]:
quiz=pd.DataFrame(quiz_table_data)

In [28]:
quiz.to_csv("EGYPT.csv",index=False)

In [30]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_12_2025_17_32_14'